In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.externals import joblib
from nltk.corpus import stopwords
import preprocessor as p
import string
from nltk.stem import WordNetLemmatizer

In [2]:
lemma = WordNetLemmatizer()
p.set_options(p.OPT.URL, p.OPT.MENTION, p.OPT.RESERVED, p.OPT.EMOJI, p.OPT.SMILEY, p.OPT.NUMBER)
stops = set(stopwords.words('english'))
replace_punc = string.maketrans(string.punctuation, ' ' * len(string.punctuation))

def preprocess(post):
    post = post.encode('ascii', errors='ignore').decode('utf-8') # Remove non ascii characters
    filtered = p.clean(post).lower() # Clean Twitter entities and make lower case
    filtered = str(filtered).translate(replace_punc) # Remove punctuation
    filtered = p.clean(filtered)
    filtered = ' '.join([lemma.lemmatize(word) for word in filtered.split() if (word not in stops) and (len(word) > 3)])
    return(filtered)

In [21]:
dataframe = pd.read_csv('dataset.csv', sep=',', header=None)
text = dataframe[0]
category = dataframe[1]
vectorizer = CountVectorizer(preprocessor=preprocess)

In [4]:
X = vectorizer.fit_transform(text).tocoo(copy=False)

In [6]:
X = pd.DataFrame({'index': X.row, 'col': X.col, 'data': X.data}
                 )[['index', 'col', 'data']].sort_values(['index', 'col']
                 ).reset_index(drop=True)

In [50]:
repeatedCategories = pd.DataFrame(category)
repeatedCategories['rep'] = X['index'].value_counts()
repeatedCategories['rep'] = repeatedCategories['rep'].fillna(0).astype(int)

In [55]:
repeatedCategories = repeatedCategories.loc[np.repeat(category.index.values, repeatedCategories.rep)].drop('rep', axis = 1)

In [58]:
naiveBayes = GaussianNB()
model = naiveBayes.fit(X, repeatedCategories.values)

/Library/Python/2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


TypeError: unsupported operand type(s) for -: 'numpy.ndarray' and 'numpy.ndarray'